文章已开源，欢迎star
https://github.com/liaokongVFX/LangChain-Chinese-Getting-Started-Guide

#装包以及初始化

In [2]:
!pip install langchain openai google-search-results unstructured chromadb pinecone-client youtube-transcript-api pytube


Note: you may need to restart the kernel to use updated packages.



Usage:   
  d:\TOMO.Project\langchain\venv\Scripts\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  d:\TOMO.Project\langchain\venv\Scripts\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  d:\TOMO.Project\langchain\venv\Scripts\python.exe -m pip install [options] [-e] <vcs project url> ...
  d:\TOMO.Project\langchain\venv\Scripts\python.exe -m pip install [options] [-e] <local project path> ...
  d:\TOMO.Project\langchain\venv\Scripts\python.exe -m pip install [options] <archive url/path> ...

no such option: -u


In [1]:
import os
# os.environ["OPENAI_API_KEY"] = 'sk-xxx'
import textwrap
from IPython.display import display
from IPython.display import Markdown

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

#使用 LangChain 完成一次问答

In [ ]:
""" 調用 openAI 的寫法 
from langchain.llms import OpenAI
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",max_tokens=1024)
llm("怎麼評價人工智能")
"""

# gemini 生成式 AI 模型
import google.generativeai as genai
genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel(model_name = "gemini-pro")
prompt_parts = ["寫一個Python函數並向我解釋一下"]
response = model.generate_content(prompt_parts)
print(response.text)
# to_markdown(response.text) #這樣無法輸出??

# gemini 對話模型
from langchain_google_genai import ChatGoogleGenerativeAI
chat_model = ChatGoogleGenerativeAI(model="gemini-pro")
result = chat_model.invoke("保持健康的最佳做法是什麼？")
to_markdown(result.content)

#通过 Google 搜索并返回答案

In [3]:
# import os
# os.environ["SERPAPI_API_KEY"] = ''

In [5]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.agents import AgentType

# 加载 OpenAI 模型
# llm = OpenAI(temperature=0,max_tokens=2048)

 # 加载 serpapi 工具
tools = load_tools(["serpapi"])
# 工具加载后都需要初始化，verbose 参数为 True，会打印全部的执行详情
# 注意！！！使用的是 chat_model
agent = initialize_agent(tools, chat_model, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 运行 agent
agent.run("What's the date today? What great events have taken place today in history? Please answer in Traditional Chinese.")



> Entering new AgentExecutor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting.


I need to find the date today and search for any significant events that occurred on this day in history.
Action: Search
Action Input: Today's date in Traditional Chinese
Observation: Saturday, February 24, 2024
Thought:Now that I have today's date, I can search for significant events that took place on this day in history.
Action: Search
Action Input: Significant events that occurred on February 24 in history in Traditional Chinese
Observation: ['The Lantern Festival also called Shangyuan Festival is a Chinese traditional festival celebrated on the fifteenth day of the first month in the lunisolar ...', 'During the Tang Dynasty (618 – 907 AD), the Lantern Festival became an important cultural event that was celebrated by people from all walks of life as a time ...', 'Chinese Lantern Festival (Yuan Xiao Jie) falls on February 24, 2024. It is a festive day and the end of the Chinese New Year (the Spring ...', '484 – King Huneric of the Vandals replaces Nicene bishops with Arian ones, an

'元宵節又稱上元節，是中國傳統節日之一，自古以來就以熱鬧喜慶的特色，被視為華人的狂歡節。'

#对超长文本进行总结

In [6]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI

# 导入文本
# loader = UnstructuredFileLoader("/content/sample_data/data/lg_test.txt")
loader = UnstructuredFileLoader("./data/lg_test.txt")
# 将文本转成 Document 对象
document = loader.load()
print(f'documents:{len(document)}')

# 初始化文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)

# 切分文本
split_documents = text_splitter.split_documents(document)
print(f'documents:{len(split_documents)}')

# 加载 llm 模型,不用 openAI
# llm = OpenAI(max_tokens=1500)
# 改用 gemini
model.max_tokens=1500
# 创建总结链
chain = load_summarize_chain(model, chain_type="refine", verbose=True)

# 执行总结链，（为了快速演示，只总结前5段）
chain.run(split_documents[:5])

documents:1
documents:319


ValidationError: 2 validation errors for LLMChain
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)
llm
  instance of Runnable expected (type=type_error.arbitrary_type; expected_arbitrary_type=Runnable)

#构建本地知识库问答机器人

In [6]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import OpenAI,VectorDBQA
from langchain.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA

# 加载文件夹中的所有txt类型的文件
loader = DirectoryLoader('./data/', glob='**/*.txt')
# 将数据转成 document 对象，每个文件会作为一个 document
documents = loader.load()

# 初始化加载器
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
# 切割加载的 document
split_docs = text_splitter.split_documents(documents)

# 初始化 openai 的 embeddings 对象
embeddings = OpenAIEmbeddings()
# 将 document 通过 openai 的 embeddings 对象计算 embedding向量信息并临时存入 Chroma 向量数据库，用于后续匹配查询
docsearch = Chroma.from_documents(split_docs, embeddings)

# 创建问答对象
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch,return_source_documents=True)
# 进行问答
result = qa({"query": "科大讯飞今年第一季度收入是多少？"})
print(result)

FileNotFoundError: Directory not found: '/content/sample_data/data/'

# 构建向量索引数据库

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

import pinecone

# 初始化 pinecone
pinecone.init(
  api_key="你的api key",
  environment="你的Environment"
)

loader = DirectoryLoader('/content/sample_data/data/', glob='**/*.txt')
# 将数据转成 document 对象，每个文件会作为一个 document
documents = loader.load()

# 初始化加载器
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# 切割加载的 document
split_docs = text_splitter.split_documents(documents)

# 初始化 openai 的 embeddings 对象
embeddings = OpenAIEmbeddings()

index_name="liaokong-test"
# 持久化数据
# docsearch = Pinecone.from_texts([t.page_content for t in split_docs], embeddings, index_name=index_name)

# 加载数据
docsearch = Pinecone.from_existing_index(index_name,embeddings)

query = "科大讯飞今年第一季度收入是多少？"
docs = docsearch.similarity_search(query, include_metadata=True)

llm = OpenAI(temperature=0)
chain = load_qa_chain(llm, chain_type="stuff", verbose=True)
chain.run(input_documents=docs, question=query)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

科大讯飞前三季营收曝光，用AI赋能企业转型推出了多款SaaS产品

而在十月二十八日晚上，在一份关于科技的报告中，复旦讯飞公布了2022年第三季度的财报。据财报，今年第一季度，科技讯飞的收入为126.5亿人民币，较上年同期增加16.5%。该公司在应对新冠肺炎疫情的过程中，在其核心产业领域，持续取得了较好的发展。

据该份财报，截至今年第三季度，科技讯飞的营运资金净流入率较去年增加百分之三十二，虽然受到了严峻的经济形势影响，但科技讯飞的营运资金和营业收入仍维持着强劲的发展势头。

虽然收入增加了不少，但在这一点上，科大讯飞的高层也坦言，与过去几个财季相比，第一个财季的收入增加速度明显慢了下来，这其中最重要的一个因素，便是由于第一个财季受到了病毒的冲击，导致了国内不少工程进度推迟。在讯飞的智能教育商业计划中，第三季度有10个左右的延期，合约金额接近16个。另外，安徽省方面，为强化对省内规划工程的协调，在第三季度内，大部分大型工程的投标都已经停止，这也是造成今年第三季度，科大讯飞收入的增幅低于预计的主要因素。

虽然收入增加了不少，但在这一点上，科大讯飞的高层也坦言，与过去几个财季相比，第一个财季的收入增加速度明显慢了下来，这其中最重要的一个因素，便是由于第一个财季受到了病毒的冲击，导致了国内不少工程进度推迟。在讯飞的智能教育商业计划中，第三季度有10个左右的延期，合约金额接近16个。另外，安徽省方面，为强化对省内规划工程的协调，在第三季度内，大部分大型工程的投标都已经停止，这也是造成今年第三季度，科大讯飞收入的增幅低于预计的主要因素。

虽然收入增加了不少，但在这一点上，科大讯飞的高层也坦言，与过去几个财季相比，第一个财季的收入增加速度明显

' 根据财报，今年第一季度，科技讯飞的收入为126.5亿人民币。'

#使用GPT3.5模型构建油管频道问答机器人


In [ ]:
import os

from langchain.document_loaders import YoutubeLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ChatVectorDBChain, ConversationalRetrievalChain

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
  ChatPromptTemplate,
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate
)

# 加载 youtube 频道
loader = YoutubeLoader.from_youtube_channel('https://www.youtube.com/watch?v=Dj60HHy-Kqk')
# 将数据转成 document
documents = loader.load()

# 初始化文本分割器
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=20
)

# 分割 youtube documents
documents = text_splitter.split_documents(documents)

# 初始化 openai embeddings
embeddings = OpenAIEmbeddings()

# 将数据存入向量存储
vector_store = Chroma.from_documents(documents, embeddings)
# 通过向量存储初始化检索器
retriever = vector_store.as_retriever()

system_template = """
Use the following context to answer the user's question.
If you don't know the answer, say you don't, don't try to make it up. And answer in Chinese.
-----------
{context}
-----------
{chat_history}
"""

# 构建初始 messages 列表，这里可以理解为是 openai 传入的 messages 参数
messages = [
  SystemMessagePromptTemplate.from_template(system_template),
  HumanMessagePromptTemplate.from_template('{question}')
]

# 初始化 prompt 对象
prompt = ChatPromptTemplate.from_messages(messages)


# 初始化问答链
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0.1,max_tokens=2048),retriever,qa_prompt=prompt)


chat_history = []
while True:
  question = input('问题：')
  # 开始发送问题 chat_history 为必须参数,用于存储对话历史
  result = qa({'question': question, 'chat_history': chat_history})
  chat_history.append((question, result['answer']))
  print(result['answer'])

问题：这个视频讲了什么？
这个视频是关于 Unreal Engine 5.2 的新实验功能的展示。在视频中介绍了一些新特性，如植被渲染、物理模拟、流体模拟、材质框架等。同时，还展示了一些环境场景的探索和汽车驾驶的游戏体验。视频还介绍了一些可用于生成可编程内容的实验工具。
问题：对环境场景的探索有哪些？
视频中展示了一个可编程的环境场景，包括树木、岩石、雾、虫子、鸟类等，同时还有一个四平方公里的区域可以探索，漫游和越野驾驶。视频还展示了汽车驾驶的游戏体验和实验工具的使用。


KeyboardInterrupt: ignored

#用 OpenAI 连接万种工具

In [ ]:
import os
os.environ["ZAPIER_NLA_API_KEY"] = ''

In [ ]:
from langchain.llms import OpenAI
from langchain.agents import initialize_agent
from langchain.agents.agent_toolkits import ZapierToolkit
from langchain.utilities.zapier import ZapierNLAWrapper


llm = OpenAI(temperature=.3)

zapier = ZapierNLAWrapper()
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
agent = initialize_agent(toolkit.get_tools(), llm, agent="zero-shot-react-description", verbose=True)

# 我们可以通过打印的方式看到我们都在 Zapier 里面配置了哪些可以用的工具
for tool in toolkit.get_tools():
  print (tool.name)
  print (tool.description)
  print ("\n\n")

Gmail: Find Email
A wrapper around Zapier NLA actions. The input to this tool is a natural language instruction, for example "get the latest email from my bank" or "send a slack message to the #general channel". Each tool will have params associated with it that are specified as a list. You MUST take into account the params when creating the instruction. For example, if the params are ['Message_Text', 'Channel'], your instruction should be something like 'send a slack message to the #general channel with the text hello world'. Another example: if the params are ['Calendar', 'Search_Term'], your instruction should be something like 'find the meeting in my personal calendar at 3pm'. Do not make up params, they will be explicitly specified in the tool description. If you do not have enough information to fill in the params, just say 'not enough information provided in the instruction, missing <param>'. If you get a none or null response, STOP EXECUTION, do not try to another tool!This too

In [ ]:
agent.run('请用中文总结最后一封"*********@qq.com"发给我的邮件。并将总结发送给"*********@qq.com"')

# 一些有意思的小Tip

##执行多个chain

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

# location 链
llm = OpenAI(temperature=1)
template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)
location_chain = LLMChain(llm=llm, prompt=prompt_template)

# meal 链
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

# 通过 SimpleSequentialChain 串联起来，第一个答案会被替换第二个中的user_meal，然后再进行询问
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)
review = overall_chain.run("Rome")



> Entering new SimpleSequentialChain chain...
Pasta alla Carbonara.

Ingredients:
- 2 cloves of garlic, minced 
- 2 tablespoons of olive oil 
- 2 cups of cooked spaghetti pasta 
- 4 slices of thick-cut bacon, cooked and chopped 
- 2 eggs, lightly beaten 
- 1/2 cup of Parmesan cheese, grated 
- Salt and black pepper to taste 

Directions:
1. In a large skillet, heat the olive oil over medium heat and add the garlic, stirring until it is fragrant (about 2 minutes).
2. Push the garlic to the side of the pan and add the cooked spaghetti pasta, stirring it around until all of the strands are coated with garlic and oil.
3. Turn off the heat, add the cooked bacon and Parmesan cheese, and stir until combined.
4. Add the beaten eggs and mix until evenly combined.
5. Season to taste with salt and pepper, and serve warm. Enjoy!

> Finished chain.


##结构化输出

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(model_name="text-davinci-003")

# 告诉他我们生成的内容需要哪些字段，每个字段类型式啥
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# 初始化解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 生成的格式提示符
# {
#	"bad_string": string  // This a poorly formatted user input string
#	"good_string": string  // This is your response, a reformatted response
#}
format_instructions = output_parser.get_format_instructions()

template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

# 讲我们的格式描述嵌入到 prompt 中去，告诉 llm 我们需要他输出什么样格式的内容
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="welcom to califonya!")
llm_output = llm(promptValue)

# 使用解析器进行解析生成的内容
output_parser.parse(llm_output)

{'bad_string': 'welcom to califonya!', 'good_string': 'Welcome to California!'}

##爬取网页并输出JSON数据

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMRequestsChain, LLMChain

llm = OpenAI(model_name="gpt-3.5-turbo", temperature=0)

template = """在 >>> 和 <<< 之间是网页的返回的HTML内容。
网页是新浪财经A股上市公司的公司简介。
请抽取参数请求的信息。

>>> {requests_result} <<<
请使用如下的JSON格式返回数据
{{
  "company_name":"a",
  "company_english_name":"b",
  "issue_price":"c",
  "date_of_establishment":"d",
  "registered_capital":"e",
  "office_address":"f",
  "Company_profile":"g"

}}
Extracted:"""

prompt = PromptTemplate(
    input_variables=["requests_result"],
    template=template
)

chain = LLMRequestsChain(llm_chain=LLMChain(llm=llm, prompt=prompt))
inputs = {
  "url": "https://vip.stock.finance.sina.com.cn/corp/go.php/vCI_CorpInfo/stockid/600519.phtml"
}

response = chain(inputs)
print(response['output'])

{
  "company_name":"贵州茅台酒股份有限公司",
  "company_english_name":"Kweichow Moutai Co.,Ltd.",
  "issue_price":"31.39",
  "date_of_establishment":"1999-11-20",
  "registered_capital":"125620万元(CNY)",
  "office_address":"贵州省仁怀市茅台镇",
  "Company_profile":"公司是根据贵州省人民政府黔府函〔1999〕291号文,由中国贵州茅台酒厂有限责任公司作为主发起人,联合贵州茅台酒厂技术开发公司、贵州省轻纺集体工业联社、深圳清华大学研究院、中国食品发酵工业研究院、北京市糖业烟酒公司、江苏省糖烟酒总公司、上海捷强烟草糖酒(集团)有限公司于1999年11月20日共同发起设立的股份有限公司。经中国证监会证监发行字[2001]41号文核准并按照财政部企[2001]56号文件的批复,公司于2001年7月31日在上海证券交易所公开发行7,150万(其中,国有股存量发行650万股)A股股票。主营业务：贵州茅台酒系列产品的生产与销售,饮料、食品、包装材料的生产与销售,防伪技术开发;信息产业相关产品的研制和开发等。"
}


##自定义工具

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

llm = OpenAI(temperature=0)

# 初始化搜索链和计算链
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)

# 生成一个功能列表，指明这个 agent 里面都有哪些可用工具
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    )
]

# 初始化 agent
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 执行 agent
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Search
Action Input: "Leo DiCaprio girlfriend"
Observation: Leonardo DiCaprio has split from girlfriend Camila Morrone. Getty. The Titanic actor hasn't been in a relationship with a woman over the age of ...
Thought: I need to find out Camila Morrone's age
Action: Search
Action Input: "Camila Morrone age"
Observation: 25 years
Thought: I now know the age and can calculate the power
Action: Calculator
Action Input: 25^0.43

> Entering new LLMMathChain chain...
25^0.43
```python
import math
print(math.pow(25, 0.43))
```

Answer: 3.991298452658078

> Finished chain.

Observation: Answer: 3.991298452658078

Thought: I now know the final answer
Final Answer: Camila Morrone's age raised to the 0.43 power is 3.991298452658078.

> Finished chain.


"Camila Morrone's age raised to the 0.43 power is 3.991298452658078."

##使用Memory实现一个带记忆的对话机器人

In [ ]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0)

# 初始化 MessageHistory 对象
history = ChatMessageHistory()

# 给 MessageHistory 对象添加对话内容
history.add_ai_message("你好！")
history.add_user_message("中国的首都是哪里？")

# 执行对话
ai_response = chat(history.messages)
print(ai_response)

content='中国的首都是北京。' additional_kwargs={}
